# Tutorial: VisionKG - A Data-Centric Way to Train your own Obejct Detection Models

In [ ]:
# Your Local Data from Annotator
local_annotation_path = '/home/mb/YOLO/dellData/annotation.json'
local_data_path = '/home/mb/YOLO/dellData/'

# where to save your data
output_image_directory = '/home/mb/YOLO/data/processedDell/images/train/'
output_annotation_directory = '/home/mb/YOLO/data/processedDell/annotations/'

# your all-in-one json file
dell_annotations = '/home/mb/YOLO/data/processedDell/annotations//annotations.json'

In [ ]:
import json
import os
import shutil

def process_coco_annotations(coco_file_path, output_dir, local_data_path, output_annotation_directory):
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(output_annotation_directory, exist_ok=True)
    with open(coco_file_path, 'r') as f:
        coco_data = json.load(f)

    old_id_to_new_id = {cat['id']: i+1 for i, cat in enumerate(sorted(coco_data['categories'], key=lambda x: x['id']))}

    for cat in coco_data['categories']:
        cat['id'] = old_id_to_new_id[cat['id']]

    for ann in coco_data['annotations']:
        ann['category_id'] = old_id_to_new_id[ann['category_id']]

    for img in coco_data['images']:
        old_image_path = local_data_path + '/' + img['file_name'] 
        new_image_path = os.path.join(output_dir, os.path.basename(old_image_path))

        if os.path.exists(old_image_path):
            shutil.copy(old_image_path, new_image_path)
        else:
            print(f" Warning: {old_image_path} is not exited.")

        img['image_path'] = new_image_path
        img['file_name'] = os.path.basename(new_image_path)

    output_json_path = os.path.join(output_annotation_directory, "annotations.json")
    with open(output_json_path, 'w') as f:
        json.dump(coco_data, f, indent=4)

    print(f"Process done! Annotations have been saved to: {output_json_path}")
    print(f"All the images have been saved to: {output_dir}")
    
    return coco_data
processed_dell_data = process_coco_annotations(local_annotation_path, output_image_directory, local_data_path, output_annotation_directory)

dell_categories = [i['name'] for i in processed_dell_data['categories']]
print(dell_categories)

# 1. QuickView of VisionKG

## 1.2 Query a Dataset as YOU need via VisionKG

In [ ]:
!python -m pip install git+https://github.com/cqels/vision.git --force
# path to your yolo directory
%cd /home/mb/YOLO/

# 2. Object Detection in Practice starting from VisionKG

## 2.1 Prepare and set parameters for training and evaluation

In [ ]:
from os.path import join as opj
from shutil import copy
from torch_model_zoo import utils

path_to_anno_mixedDatasets = dell_annotations
filter_cat_nms = dell_categories
params = utils.prepare_for_training(path_to_anno_mixedDatasets, processed_dell_data, existed_data=output_image_directory, filter_cat_nms=filter_cat_nms)
    
nms_categories = params['CAT_NMS']
num_categories = len(nms_categories)

In [ ]:
# move data

annotation_path = output_annotation_directory
# I want to let img_path is the father folder of output_image_directory
img_path = opj(output_image_directory, '..')

anno_files = ['train.json','val.json', 'test.json']
img_folders = ['train2017', 'val2017', 'test2017']

for anno_file, img_folder in zip(anno_files, img_folders):
    anno_file_ = opj(annotation_path, anno_file)
    new_anno_file = opj(annotation_path, 'instances_' + anno_file.split('.')[0] + '2017.json')
    img_folder = opj(img_path, img_folder)
    os.makedirs(img_folder, exist_ok=True)
    with open(anno_file_, 'r') as f:
        anno = json.load(f)
    # save anno to new_anno_file
    with open(new_anno_file, 'w') as f:
        json.dump(anno, f, indent=4)
    for img in anno['images']:
        img_name = img['file_name']
        raw_img_path = opj(local_data_path, img_name)
        dst_img_path = opj(img_folder, img_name)
        copy(raw_img_path, dst_img_path)
    print(f"Images in {raw_img_path} have been moved to {dst_img_path}")

In [ ]:
# edit config file
import yaml
config_path = 'yolo/config/dataset/dellData.yaml'
with open(config_path, 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    config['class_num'] = num_categories
    config['class_list'] = nms_categories
# save config
with open(config_path, 'w') as f:
    yaml.dump(config, f, default_flow_style=None, sort_keys=False)

## 2.2 Data-Playground

In [ ]:
if num_categories > 4:
  cat_nms = nms_categories[0:4]
else:
  cat_nms = nms_categories
utils.show_annotation(path_to_anno_mixedDatasets, cat_nms, show_num=6)
utils.show_cat_distribution(path_to_anno_mixedDatasets, cat_nms)

## 2.2 Perform Training & Evaluation on your chosen Object Detection tool

In [10]:
# Training based on the queried MixedDataset
# For more params-setting, please check:
# https://yolo-docs.readthedocs.io/en/latest/?badge=latest

%env CUDA_VISIBLE_DEVICES=0

%run yolo/lazy.py task=train task.data.batch_size=8 model=v9-c dataset=dellData task.epoch=10 weight=True

env: CUDA_VISIBLE_DEVICES=0


[02/10/25 00:07:36] INFO     📄 Created log folder: runs/train/v9-dev                          ]8;id=665275;file:///jicheng_workspace/jicheng_notebook/YOLO/yolo/utils/logging_utils.py\logging_utils.py]8;;\:]8;id=110420;file:///jicheng_workspace/jicheng_notebook/YOLO/yolo/utils/logging_utils.py#324\324]8;;\

                    INFO     📈 Enable Model EMA                                                  ]8;id=992960;file:///jicheng_workspace/jicheng_notebook/YOLO/yolo/utils/model_utils.py\model_utils.py]8;;\:]8;id=859731;file:///jicheng_workspace/jicheng_notebook/YOLO/yolo/utils/model_utils.py#43\43]8;;\

save_path:  runs/train/v9-dev


[02/10/25 00:07:36] INFO     ⚡ Using 16bit Automatic Mixed Precision (AMP)            ]8;id=675844;file:///root/anaconda3/envs/yolodell/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/accelerator_connector.py\accelerator_connector.py]8;;\:]8;id=923086;file:///root/anaconda3/envs/yolodell/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/accelerator_connector.py#520\520]8;;\

...

INFO: TPU available: False, using: 0 TPU cores


[2025-02-10 00:07:36,573][lightning.pytorch.utilities.rank_zero][INFO] - TPU available: False, using: 0 TPU cores


                    INFO     ⚡ HPU available: False, using: 0 HPUs                                    ]8;id=346267;file:///root/anaconda3/envs/yolodell/lib/python3.9/site-packages/lightning/pytorch/trainer/setup.py\setup.py]8;;\:]8;id=835594;file:///root/anaconda3/envs/yolodell/lib/python3.9/site-packages/lightning/pytorch/trainer/setup.py#169\169]8;;\

INFO: HPU available: False, using: 0 HPUs


[2025-02-10 00:07:36,578][lightning.pytorch.utilities.rank_zero][INFO] - HPU available: False, using: 0 HPUs
{'name': 'v9-dev', 'task': {'validation': {'task': 'validation', 'data': {'batch_size': 32, 'image_size': '${image_size}', 'cpu_num': '${cpu_num}', 'shuffle': False, 'pin_memory': True, 'data_augment': {}, 'dynamic_shape': False}, 'nms': {'min_confidence': 0.0001, 'min_iou': 0.7, 'max_bbox': 1000}}, 'task': 'train', 'epoch': 10, 'data': {'batch_size': 8, 'image_size': '${image_size}', 'cpu_num': '${cpu_num}', 'shuffle': True, 'pin_memory': True, 'data_augment': {'HorizontalFlip': 0.5}}, 'optimizer': {'type': 'SGD', 'args': {'lr': 1e-07, 'weight_decay': 0.0005, 'momentum': 0.937, 'nesterov': True}}, 'loss': {'objective': {'BCELoss': 0.5, 'BoxLoss': 7.5, 'DFLoss': 1.5}, 'aux': 0.25, 'matcher': {'iou': 'CIoU', 'topk': 10, 'factor': {'iou': 6.0, 'cls': 0.5}}}, 'scheduler': {'type': 'LinearLR', 'warmup': {'epochs': 3.0}, 'args': {'total_iters': '${task.epoch}', 'start_factor': 1, 'en

In [ ]:
# import subprocess

# command  = "CUDA_VISIBLE_DEVICES=0 python yolo/lazy.py task=inference name=test device=cuda model=v9-c dataset=dellData task.nms.min_confidence=0.1 task.fast_inference=onnx task.data.source=data/processedDell/images/test2017/ weight='/jicheng_workspace/jicheng_notebook/YOLO/runs/train/v9-dev/YOLO/kf65nxvs/checkpoints/yolo.ckpt'"

# subprocess.run(command, shell=True, check=True)

In [ ]:
# %run yolo/lazy.py task=inference name=test device=cuda model=v9-c dataset=dellData \
# task.nms.min_confidence=0.1 task.fast_inference=onnx \
# task.data.source=data/processedDell/images/test2017/ \
# weight="'/jicheng_workspace/jicheng_notebook/YOLO/runs/train/v9-dev/YOLO/5t6dcono/checkpoints/epoch=9-step=120.ckpt'"